# Algorithmic Accompaniment Generator (ver 2.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2023

***

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install all dependencies (run only once per session)
!git clone https://github.com/asigalov61/tegridy-tools
!apt install fluidsynth

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os
import copy
import math
import statistics
import random

from collections import Counter

from tqdm import tqdm

if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

print('Loading TMIDIX module...')
os.chdir('/content/tegridy-tools/tegridy-tools')

import TMIDIX
import HaystackSearch
from midi_to_colab_audio import midi_to_colab_audio

from joblib import Parallel, delayed, parallel_config

from IPython.display import Audio, display

print('Done!')

os.chdir('/content/')
print('Enjoy! :)')

# (DOWNLOAD SAMPLE MIDI DATASETS)

## NOTE: Do not download/process ASAP MIDI dataset if you are on a free-tier Google Colab because all three MIDI datasets will not fit into free-tier RAM

In [ ]:
# @title Download and unzip Mono Melodies Piano Violin MIDI Dataset
%cd /content/Dataset
!wget https://github.com/asigalov61/Tegridy-MIDI-Dataset/raw/master/Mono-Melodies/Piano-Violin/Mono-Melodies-Piano-Violin-CC-BY-NC-SA.zip.001
!wget https://github.com/asigalov61/Tegridy-MIDI-Dataset/raw/master/Mono-Melodies/Piano-Violin/Mono-Melodies-Piano-Violin-CC-BY-NC-SA.zip.002
!cat Mono-Melodies-Piano-Violin-CC-BY-NC-SA.zip* > Mono-Melodies-Piano-Violin-CC-BY-NC-SA.zip
!unzip Mono-Melodies-Piano-Violin-CC-BY-NC-SA.zip
!rm Mono-Melodies-Piano-Violin-CC-BY-NC-SA.zip
%cd /content/

In [ ]:
# @title Download and unzip Tegridy Piano MIDI Dataset
%cd /content/Dataset
!wget https://github.com/asigalov61/Tegridy-MIDI-Dataset/raw/master/Tegridy-Piano-CC-BY-NC-SA.zip
!unzip Tegridy-Piano-CC-BY-NC-SA.zip
!rm Tegridy-Piano-CC-BY-NC-SA.zip
%cd /content/

In [ ]:
# @title Download ASAP MIDI Dataset
%cd /content/Dataset

!git clone --depth 1 https://github.com/fosfrancesco/asap-dataset
%cd /content/

# (FILE LIST)

# (PROCESS MIDIs)

In [ ]:
#@title TMIDIX MIDI Processor

print('=' * 70)
print('Loading TMIDIX MIDI Processor...')
print('=' * 70)

def TMIDIX_MIDI_Processor(midi_file):

    melody_chords = []

    try:

        fn = os.path.basename(midi_file)

        # Filtering out GIANT4 MIDIs
        file_size = os.path.getsize(midi_file)

        if file_size <= 1000000:

          #=======================================================
          # START PROCESSING

          # Convering MIDI to ms score with MIDI.py module
          score = TMIDIX.midi2single_track_ms_score(open(midi_file, 'rb').read(), recalculate_channels=False)

          # INSTRUMENTS CONVERSION CYCLE
          events_matrix = []
          itrack = 1
          patches = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

          while itrack < len(score):
              for event in score[itrack]:
                  if event[0] == 'note' or event[0] == 'patch_change':
                      events_matrix.append(event)
              itrack += 1

          events_matrix.sort(key=lambda x: x[1])

          events_matrix1 = []

          for event in events_matrix:
                  if event[0] == 'patch_change':
                        patches[event[2]] = event[3]

                  if event[0] == 'note':
                        event.extend([patches[event[3]]])

                        if events_matrix1:
                            if (event[1] == events_matrix1[-1][1]):
                                if ([event[3], event[4]] != events_matrix1[-1][3:5]):
                                    events_matrix1.append(event)
                            else:
                                events_matrix1.append(event)

                        else:
                            events_matrix1.append(event)

        if len(events_matrix1) > 0:
            if min([e[1] for e in events_matrix1]) >= 0 and min([e[2] for e in events_matrix1]) >= 0:

                #=======================================================
                # PRE-PROCESSING

                # checking number of instruments in a composition
                instruments_list = list(set([y[3] for y in events_matrix1]))

                if len(events_matrix1) > 0:

                    #===================================
                    # ORIGINAL COMPOSITION
                    #===================================

                    # Adjusting timings

                    for e in events_matrix1:
                      e[1] = int(e[1] / 16)
                      e[2] = int(e[2] / 16)

                    # Sorting by patch, pitch, then by start-time

                    events_matrix1.sort(key=lambda x: x[6])
                    events_matrix1.sort(key=lambda x: x[4], reverse=True)
                    events_matrix1.sort(key=lambda x: x[1])

                    #=======================================================
                    # FINAL PROCESSING

                    #=======================================================
                    # MAIN PROCESSING CYCLE
                    #=======================================================

                    tr_chords = []

                    for tr in range(-6, 6):

                      pe = events_matrix1[0]

                      notes = []

                      for e in events_matrix1:

                        time = max(0, min(255, (e[1] - pe[1])))
                        dur = max(0, min(255, e[2]))
                        cha = max(0, min(15, e[3]))
                        ptc = max(1, min(127, e[4]))+tr

                        if cha in [0, 1, 2, 4, 5, 10, 11]:
                          cha = 0
                        if cha in [3, 6, 7, 8]:
                          cha = 3

                        if cha != 9:

                            notes.append([time, ptc])
                            pe = e

                      chords = []
                      cho = []

                      for n in notes:

                        if n[0] == 0:

                          cho.append(n[1])
                        else:
                          if len(cho) > 0:
                            chords.append(cho)

                          cho = []
                          cho.append(n[1])

                      if len(cho) > 0:
                        chords.append(cho)

                      #==============================

                      tr_chords.append(chords)

                    return tr_chords

    except:
      return None

print('Done!')
print('=' * 70)

In [ ]:
#@title Save file list
###########

print('=' * 70)
print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/Dataset"

# os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames if file.split('.')[-1] == 'mid']
print('=' * 70)

if not filez:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)

else:
  print('Randomizing file list...')
  random.shuffle(filez)
  print('Done!')
  print('=' * 70)
  print('Total files:', len(filez))
  print('=' * 70)

In [ ]:
#@title Process MIDIs with TMIDIX MIDI processor

print('=' * 70)
print('TMIDIX MIDI Processor')
print('=' * 70)
print('Starting up...')
print('=' * 70)

###########

melody_chords_f = []

print('Processing MIDI files. Please wait...')
print('=' * 70)

for i in tqdm(range(0, len(filez), 16)):

  with parallel_config(backend='threading', n_jobs=16, verbose = 0):

    output = Parallel()(delayed(TMIDIX_MIDI_Processor)(f) for f in filez[i:i+16])

    for o in output:

        if o is not None:
          for oo in o:
            melody_chords_f.append(oo)

print('=' * 70)
print('Extracting melodies...')
print('=' * 70)

melodies = []

for m in tqdm(melody_chords_f):
  mel = []
  for mm in m:
    mel.append(mm[0] % 12)
  melodies.append(mel)

print('Done!')
print('=' * 70)

# (SAVE/LOAD PROCESSED MIDIs)

In [ ]:
# @title Save processed MIDIs
TMIDIX.Tegridy_Any_Pickle_File_Writer([melody_chords_f, melodies], '/content/Processed_MIDIs')


In [ ]:
# @title Load processed MIDIs
melody_chords_f, melodies = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/Processed_MIDIs')
print('Done!')

# (GENERATE ACCOMPANIMENT)

In [ ]:
# @title Load source MIDI
full_path_to_source_MIDI = "/content/tegridy-tools/tegridy-tools/seed-melody.mid" # @param {type:"string"}
melody_transpose_value = 0 # @param {type:"slider", min:-6, max:6, step:1}

#@markdown Option below increases density and dynamics of the accompaniment but it may not work well on some melodies.

multiply_melody_timings = True # @param {type:"boolean"}

#===============================================================================

def multiply_melody_times(melody_notes, max_dur=34, min_dur=11):

    notes = copy.deepcopy(melody_notes)

    multiplied_melody_notes = []
    nc = 0

    for nn in notes[:-1]:

      if nn[1] > max_dur:

        quotient, remainder = divmod(nn[1], max_dur)
        result = []
        n = copy.deepcopy(nn)
        n[1] = max_dur-1
        n.append(1)
        result.append(n)
        for i in range(quotient-1):
          n = copy.deepcopy(nn)
          n[0] = max_dur
          n[1] = max_dur-1
          n.append(0)
          result.append(n)
        if remainder >= min_dur:
          n = copy.deepcopy(nn)
          n[0] = max_dur-1
          n[1] = remainder
          n.append(0)
          result.append(n)
          notes[nc+1][0] = copy.deepcopy(notes[nc+1][0]-(quotient*max_dur))

        else:
          notes[nc+1][0] = copy.deepcopy(notes[nc+1][0]-((quotient-1)*max_dur))

        multiplied_melody_notes.extend(result)

      else:

        n = copy.deepcopy(nn)
        n.append(1)
        multiplied_melody_notes.append(n)

      nc += 1

    max_dur = nn[1]
    n = copy.deepcopy(notes[-1])
    n[0] = copy.deepcopy(notes[-1][0]-((quotient)*max_dur)+remainder)
    n.append(1)
    multiplied_melody_notes.append(n)

    return multiplied_melody_notes

#===============================================================================

print('=' * 70)
print('Loading source MIDI file...')

# Convering MIDI to ms score with MIDI.py module
score = TMIDIX.midi2single_track_ms_score(open(full_path_to_source_MIDI, 'rb').read(), recalculate_channels=False)

# INSTRUMENTS CONVERSION CYCLE
events_matrix = []
itrack = 1
patches = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

while itrack < len(score):
    for event in score[itrack]:
        if event[0] == 'note' or event[0] == 'patch_change':
            events_matrix.append(event)
    itrack += 1

events_matrix.sort(key=lambda x: x[1])

events_matrix1 = []

for event in events_matrix:
        if event[0] == 'patch_change':
              patches[event[2]] = event[3]

        if event[0] == 'note':
              event.extend([patches[event[3]]])

              if events_matrix1:
                  if (event[1] == events_matrix1[-1][1]):
                      if ([event[3], event[4]] != events_matrix1[-1][3:5]):
                          events_matrix1.append(event)
                  else:
                      events_matrix1.append(event)

              else:
                  events_matrix1.append(event)

if len(events_matrix1) > 0:
  if min([e[1] for e in events_matrix1]) >= 0 and min([e[2] for e in events_matrix1]) >= 0:

      #=======================================================
      # PRE-PROCESSING

      # checking number of instruments in a composition
      instruments_list = list(set([y[3] for y in events_matrix1]))

      if len(events_matrix1) > 0:

          #===================================
          # ORIGINAL COMPOSITION
          #===================================

          # Adjusting timings

          for e in events_matrix1:
            e[1] = int(e[1] / 16)
            e[2] = int(e[2] / 16)

          # Sorting by patch, pitch, then by start-time

          events_matrix1.sort(key=lambda x: x[6])
          events_matrix1.sort(key=lambda x: x[4], reverse=True)
          events_matrix1.sort(key=lambda x: x[1])

          #=======================================================
          # FINAL PROCESSING

          #=======================================================
          # MAIN PROCESSING CYCLE
          #=======================================================

          tr_chords = []

          for tr in range(-6, 6):

            pe = copy.deepcopy(events_matrix1[0])
            pe[1] -= 1

            notes = []

            durs = []
            times = []

            for e in events_matrix1:

              time = max(0, min(255, (e[1] - pe[1])))
              times.append(time)
              dur = max(0, min(255, e[2]))
              durs.append(dur)
              cha = max(0, min(15, e[3]))
              ptc = max(1, min(127, e[4]))+tr

              if cha != 9 and time != 0:
                  notes.append([time, dur, 3, ptc])
                  pe = e

for n in notes:
  n[3] += melody_transpose_value

fixed_durs = []
abs_time = 0

for i in range(len(notes)-1):
  note = notes[i]
  abs_time += note[0]
  nmt = abs_time+notes[i+1][0]

  if abs_time+note[1] >= nmt:
    note_dur = nmt-abs_time-1
  else:
    note_dur = note[1]

  fixed_durs.append([note[0], note_dur, note[2], note[3]])

fixed_durs.append(notes[-1])

if multiply_melody_timings:
  melody_full = multiply_melody_times(fixed_durs, round(sum(durs) / len(durs)), min(durs))
else:
  melody_full = fixed_durs

melody_ptc = [n[3] % 12 for n in melody_full]

print('Done!')
print('=' * 70)
print('Number of melody notes:', len(melody_ptc))
print('=' * 70)
print('Done!')
print('=' * 70)

In [ ]:
# @title Generate all possible accompaniments

#@markdown Run this only once per MIDI file

#@markdown You can stop the generatio at any time to render partial results

melody_chunk_length = 8 # @param {type:"slider", min:4, max:10, step:1}
melody_chunks_overlap = 4 # @param {type:"slider", min:3, max:9, step:1}

print('=' * 70)
print('Generating all possible accompaniments...')
print('=' * 70)

def chunk_melody(melody, chunk_size, overlap=0):

    lst = melody

    # Initialize the list to hold chunks
    chunks = []

    # Use a loop to create chunks with the specified overlap
    i = 0
    while i < len(lst):
        chunks.append(lst[i:i + chunk_size])
        i += chunk_size - overlap  # Move the index by chunk_size minus overlap

        # Handle the special case for the last chunk
        if i >= len(lst) - chunk_size and i < len(lst):
            chunks.append(lst[-chunk_size:])
            break

    return chunks

def search_melodies(melody_chunk):

  acc_chunks = []

  for i in range(len(melody_chords_f)):
    mel = melodies[i]
    idx = HaystackSearch.HaystackSearch(melody_chunk, mel)
    if idx > -1:
      acc_chunks.append(melody_chords_f[i][idx:idx+melody_chunk_length])

  return acc_chunks

#===============================================================================

mel_chunks = chunk_melody(melody_ptc, melody_chunk_length, melody_chunks_overlap)

output = []

for mc in tqdm(mel_chunks):

  try:
    output.append(search_melodies(mc))

  except KeyboardInterrupt:
    break

  except Exception as e:
    print(e)
    break

print('Done!')
print('=' * 70)

In [ ]:
# @title Process and render random accompaniment

#@markdown You can re-run this cell as many times as you wish to generate unique varriations of the accompaniment

melody_MIDI_patch = 40 # @param {type:"slider", min:0, max:127, step:1}
accompaniment_MIDI_patch = 0 # @param {type:"slider", min:0, max:127, step:1}
single_notes_multiplication_value = 4 # @param {type:"slider", min:0, max:6, step:1}

acc_chunks_f = []

def transpose_pitches_to_c4(pitches):
  p1 = []
  for pp in pitches:
    p1.append([(p % 12)+60 for p in pp])

  return p1

print('=' * 70)
print('Processing random accompaniment...')

missing_chunks = 0

chunk_idx = 0

# Processing all chunks but last
for out in output[:-1]:
  if out:
    random.shuffle(out)
    max_acc_chunk = max(out, key=len)
    acc_chunks_f.append(max_acc_chunk[:-melody_chunks_overlap])
  else:
    acc_chunks_f.append([[m+60] for m in mel_chunks[chunk_idx][:-melody_chunks_overlap]])
    missing_chunks += 1

  chunk_idx += 1

# Processing last chunk
if output[-1]:
  max_acc_chunk = max(output[-1], key=len)
  acc_chunks_f.append(max_acc_chunk)
else:
  acc_chunks_f.append([[m+60 for m in mel_chunks[-1][:-melody_chunks_overlap]]])
  missing_chunks += 1

missing_chords = 0

acc_chunks_f1 = []
idx = 0
for a in acc_chunks_f:
  if a[0]:
    if max([max(aa) for aa in a]) >= 72:
      acc_chunks_f1.extend(transpose_pitches_to_c4(a))
    else:
      acc_chunks_f1.extend(a)
  else:
    acc_chunks_f1.append([[m+60] for m in mel_chunks[chunk_idx+1]])
    missing_chords += 1

  idx += 1

print('=' * 70)
print('Number of missing chunks:', missing_chunks, '/', len(output))
print('Number of missing chords:', missing_chords, '/', len(acc_chunks_f1))
print('If any chunks/chords are missing you can reduce the melody chunks length to prevent it')
print('=' * 70)
mel_acc = []

mel_idx = 0

for a in acc_chunks_f1[:len(melody_full)]:

    aaa = copy.deepcopy(a)
    aaa1 = [copy.deepcopy(melody_full[mel_idx][3])]
    if len(list(set(aaa))) == 1:
      for i in range(single_notes_multiplication_value):
        if i != 0:
          aaa1.append(aaa1[0])
          aaa1[i] -= (12*i)
      mel_acc.append(aaa1)

    if len(aaa) != 1:
      mel_acc.append(aaa)
    mel_idx += 1

print('Done!')

patches = [0] * 16
patches[0] = accompaniment_MIDI_patch
patches[3] = melody_MIDI_patch

print('=' * 70)

if len(mel_acc) != 0:

    song = mel_acc
    song_f = []

    time = 0
    dur = 0
    vel = 90
    pitch = 0
    channel = 0

    if multiply_melody_timings:

      mel_idx = 0
      mel_idx1 = 0

      for ss in song[:len(melody_full)]:

        if melody_full[mel_idx][4] == 1:
          time += melody_full[mel_idx][0] * 16
          dur = fixed_durs[mel_idx1][1] * 16
          ptc = fixed_durs[mel_idx1][3]
          channel = 3
          vel = ptc+20

          song_f.append(['note', time, dur, channel, ptc, vel, patches[channel] ])
          mel_idx1 += 1

        else:
          time += melody_full[mel_idx][0] * 16
          dur = melody_full[mel_idx][1] * 16
          ptc = melody_full[mel_idx][3]

        for s in ss:

          if ptc != s:

            channel = 0
            pitch = s
            vel = pitch

            song_f.append(['note', time, dur, channel, pitch, vel, patches[channel] ])

        mel_idx += 1
    else:

      mel_idx = 0

      for ss in song[:len(melody_full)]:

        time += melody_full[mel_idx][0] * 16
        dur = melody_full[mel_idx][1] * 16
        ptc = melody_full[mel_idx][3]
        channel = 3
        song_f.append(['note', time, dur, channel, ptc, vel, patches[channel] ])

        for s in ss[1:]:

          if ptc != s:

            channel = 0
            pitch = s
            song_f.append(['note', time, dur, channel, pitch, vel, patches[channel] ])

        mel_idx += 1

detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                          output_signature = 'Algorithmic Accompaniment',
                                                          output_file_name = '/content/Algorithmic-Accompaniment-Composition',
                                                          track_name='Project Los Angeles',
                                                          list_of_MIDI_patches=patches
                                                          )

print('=' * 70)
print('Displaying resulting composition...')
print('=' * 70)

fname = '/content/Algorithmic-Accompaniment-Composition'

midi_audio = midi_to_colab_audio(fname + '.mid')
display(Audio(midi_audio, rate=16000, normalize=False))

TMIDIX.plot_ms_SONG(song_f, plot_title=fname)

# Congrats! You did it! :)